In [1]:
# Dependencies
import matplotlib.pyplot as plt
import requests
import pandas as pd
from pprint import pprint
from datetime import datetime
import os
import csv

In [2]:
# Save config information.
url = "https://data.ca.gov/api/3/action/datastore_search?resource_id=d73ee828-c2c6-485c-91dc-c26bd9ce3991&"

# Build partial query URL
limit = 1000
year = 2021
query_url = f"{url}&limit={limit}&q={year}"
response = requests.get(query_url).json()
response

{'help': 'https://data.ca.gov/api/3/action/help_show?name=datastore_search',
 'success': True,
 'result': {'include_total': True,
  'limit': 1000,
  'q': '2021',
  'records_format': 'objects',
  'resource_id': 'd73ee828-c2c6-485c-91dc-c26bd9ce3991',
  'total_estimation_threshold': None,
  'records': [{'_id': 820,
    'OBJECTID': '820',
    'Year': '2021',
    'State': 'CA',
    'Agency': 'LRA',
    'Unit ID': 'LFD',
    'Fire Name': 'DANTONI',
    'Local Incident Number': '00024333',
    'Alarm Date': '9/11/2021 12:00:00 AM',
    'Containment Date': '9/12/2021 12:00:00 AM',
    'Cause': '14',
    'Collection Method': '6',
    'Management Objective': '1',
    'GIS Calculated Acres': '24.35467',
    'Comments': 'Linda Fire Department, LFPD 2021-2540118',
    'Complex Name': None,
    'IRWIN ID': None,
    'Fire Number (historical use)': None,
    'Complex ID': None,
    'Shape__Area': '164267.91796875',
    'Shape__Length': '1850.80460930434',
    'rank': 0.0706241},
   {'_id': 821,
    

In [3]:
df = pd.DataFrame(response['result']['records'])
df_2021 = df.loc[df['Year'] == f'{str(year)}']
columns_to_keep = ['Year', 'Alarm Date','Containment Date','Cause','GIS Calculated Acres']
columns_to_drop = [col for col in df_2021.columns if col not in columns_to_keep]
df_2021 = df_2021.drop(columns=columns_to_drop)
df_2021.head()

,Year,Alarm Date,Containment Date,Cause,GIS Calculated Acres
0,2021,9/11/2021 12:00:00 AM,9/12/2021 12:00:00 AM,14,24.35467
1,2021,9/12/2021 12:00:00 AM,9/13/2021 12:00:00 AM,10,70.8007
2,2021,10/5/2021 12:00:00 AM,10/5/2021 12:00:00 AM,14,41.59887
3,2021,8/12/2021 12:00:00 AM,8/12/2021 12:00:00 AM,14,15.82752
4,2021,5/8/2021 12:00:00 AM,5/8/2021 12:00:00 AM,14,60.39558


In [6]:
df_2021['Alarm Date'] = [dt.replace(' 12:00:00 AM', '') for dt in df_2021['Alarm Date']]
df_2021['Containment Date'] = [str(item) for item in df_2021['Containment Date']]
df_2021['Containment Date'] = [dt.replace(' 12:00:00 AM', '') for dt in df_2021['Containment Date']]
df_2021.head()

,Year,Alarm Date,Containment Date,Cause,GIS Calculated Acres
0,2021,9/11/2021,9/12/2021,14,24.35467
1,2021,9/12/2021,9/13/2021,10,70.8007
2,2021,10/5/2021,10/5/2021,14,41.59887
3,2021,8/12/2021,8/12/2021,14,15.82752
4,2021,5/8/2021,5/8/2021,14,60.39558


In [7]:
df_2021 = df_2021.rename(columns={'Cause': 'Cause #'})
df_2021['Cause'] = ''
df_2021['Cause #'] = df_2021['Cause #'].astype(int)
def map_cause_to_meaning(i):
    if i == 1:
        return 'Lightning'
    elif i == 2:
        return 'Equipment Use'
    elif i == 3:
        return 'Smoking'
    elif i == 4:
        return 'Campfire'
    elif i == 5:
        return 'Debris'
    elif i == 6:
        return 'Railroad'
    elif i == 7:
        return 'Arson'
    elif i == 8:
        return 'Playing with fire'
    elif i == 9:
        return 'Miscellaneous'
    elif i == 10:
        return 'Vehicle'
    elif i == 11:
        return 'Powerline'
    elif i == 12:
        return 'Firefighter Training'
    elif i == 13:
        return 'Non-Firefighter Training'
    elif i == 14:
        return 'Unknown / Unidentified'
    elif i == 15:
        return 'Structure'
    elif i == 16:
        return 'Aircraft'
    elif i == 18:
        return 'Escaped Prescribed Burn'
    elif i == 19:
        return 'Illegal Alien Campfire'
    else:
        return 'N/A'
df_2021['Cause'] = df_2021['Cause #'].apply(map_cause_to_meaning)
df_2021.head()

,Year,Alarm Date,Containment Date,Cause #,GIS Calculated Acres,Cause
0,2021,9/11/2021,9/12/2021,14,24.35467,Unknown / Unidentified
1,2021,9/12/2021,9/13/2021,10,70.8007,Vehicle
2,2021,10/5/2021,10/5/2021,14,41.59887,Unknown / Unidentified
3,2021,8/12/2021,8/12/2021,14,15.82752,Unknown / Unidentified
4,2021,5/8/2021,5/8/2021,14,60.39558,Unknown / Unidentified


In [14]:
df_2021.to_csv("outputs/fires_2021.csv",
                  encoding="utf-8", index=False, header=True)

In [15]:
df_2021['Alarm Date'] = pd.to_datetime(df_2021['Alarm Date'])
df_byalarm = df_2021.sort_values(by='Alarm Date').reset_index()
df_byalarm.head()

,index,Year,Alarm Date,Containment Date,Cause #,GIS Calculated Acres,Cause
0,373,2021,2021-01-07,2021-01-07,14,21.31999,Unknown / Unidentified
1,174,2021,2021-01-08,2021-01-08,14,2.265879,Unknown / Unidentified
2,359,2021,2021-01-08,2021-01-08,14,1.865689,Unknown / Unidentified
3,143,2021,2021-01-10,2021-01-21,9,1.850452,Miscellaneous
4,144,2021,2021-01-13,2021-01-21,5,18.64428,Debris


In [16]:
earliest_fire = df_byalarm['Alarm Date'][0]
earliest_fire

Timestamp('2021-01-07 00:00:00')

In [18]:
df_2021['Containment Date'] = pd.to_datetime(df_2021['Containment Date'])
df_bycontainment = df_2021.sort_values(by='Containment Date', ascending = False).reset_index()
latest_containment = df_bycontainment['Containment Date'][0]
df_bycontainment.head()

,index,Year,Alarm Date,Containment Date,Cause #,GIS Calculated Acres,Cause
0,168,2021,2021-09-07,2022-09-07,2,25.03143,Equipment Use
1,142,2021,2021-05-02,2022-05-06,2,5365.902,Equipment Use
2,293,2021,2021-08-01,2022-01-03,1,145631.8,Lightning
3,275,2021,2021-06-28,2022-01-03,14,10591.03,Unknown / Unidentified
4,140,2021,2021-09-10,2021-12-16,1,88257.13,Lightning


In [19]:
latest_containment

Timestamp('2022-09-07 00:00:00')

In [20]:
output_path = os.path.join("outputs", "timeline_dates.csv")
with open(output_path, 'w') as csvfile:

    # Initialize csv.writer
    csvwriter = csv.writer(csvfile, delimiter=',')

    # Write the first row (column headers)
    csvwriter.writerow(['Title', 'Date'])
    csvwriter.writerow(['Earliest Fire', earliest_fire])

    # Write the second row
    csvwriter.writerow(['Latest Containment', latest_containment])